# Imports

In [1]:
import torch
import transformers
from tokenizers import Tokenizer
from tokenizers import normalizers, pre_tokenizers, models, processors, decoders, trainers
from torch.utils.data import DataLoader
from datasets import load_dataset, total_allocated_bytes

import config

In [2]:
# Load wikitext-103 dataset
train_ds, val_ds, test_ds = load_dataset('wikitext', 'wikitext-103-raw-v1', split=["train", "validation", "test"])

Reusing dataset wikitext (C:\Users\sap98\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91)


# Train Tokenizer

Train a Byte Pair Encoding (BPE) tokenizer with the following characteristics:
- Splits on whitespace
- Store info about space (metaspace)
- Split digits from words

In [ ]:
# Define trainer
trainer = trainers.BpeTrainer(
    vocab_size=config.VOCAB_SIZE,
    special_tokens=["[PAD]", "[CLS]", "[SEP]", "[EOS]", "[UNK]", "[MASK]"],
    show_progress=True
)

In [4]:
# Define tokenizer
tokenizer = Tokenizer(models.BPE())

# NFKD Unicode Normalization, all lowercase
tokenizer.normalizer = normalizers.Sequence([
    normalizers.NFKD(),
    normalizers.Lowercase(),
])

# Split on whitespace and digits, and store info about space (metaspace)
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Whitespace(),
    pre_tokenizers.Metaspace(),
    pre_tokenizers.Digits(individual_digits=False)
])

# Decoding
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [EOS]",
    pair="[CLS] $A [SEP] $B:1 [EOS]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2), ("[EOS]", 3)]
)
tokenizer.decoder = decoders.BPEDecoder()

In [5]:
tokenizer.train(trainer, config.PATH_DATA_FILES)

In [6]:
# Enable padding
# Pad to longest string in batch
tokenizer.enable_padding(
    direction="right",
    pad_id=tokenizer.token_to_id("[PAD]")
)

In [7]:
files = tokenizer.model.save(config.PATH_DATA)
tokenizer.model = models.BPE.from_file(*files, unk_token="[UNK]")
tokenizer.save(config.PATH_TOKENIZER)

# Tokenize Dataset

In [9]:
def tokenize(batch):
    encoded = tokenizer.encode_batch(batch["text"])
    ids = []
    type_ids = []
    attention_mask = []
    special_tokens_mask = []

    for x in encoded:
        ids.append(x.ids)
        attention_mask.append(x.attention_mask)
    
    return {
        "ids": ids,
        "attention_mask": attention_mask,
    }

In [10]:
# Tokenize the data
ttrain_ds = train_ds.map(tokenize, batched=True, batch_size=config.BATCH_SIZE)
tval_ds = val_ds.map(tokenize, batched=True, batch_size=config.BATCH_SIZE)
ttest_ds = test_ds.map(tokenize, batched=True, batch_size=config.BATCH_SIZE)

100%|██████████| 35/35 [00:01<00:00, 33.62ba/s]


In [11]:
# Save the tokenized data to disk
ttrain_ds.save_to_disk(config.PATH_TRAIN_TOK)
tval_ds.save_to_disk(config.PATH_VAL_TOK)
ttest_ds.save_to_disk(config.PATH_TEST_TOK)

tval_ds.column_names

['attention_mask', 'ids', 'text']

# Look at stuff

In [12]:
ttrain_ds.set_format(type="np", columns=["ids", "attention_mask"])

In [13]:
ttrain_ds.column_names

['attention_mask', 'ids', 'text']

In [14]:
dataloader = DataLoader(ttrain_ds, batch_size=config.BATCH_SIZE)

In [15]:
d = iter(dataloader)
next(d)

{'attention_mask': tensor([[1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'ids': tensor([[   1,    3,    0,  ...,    0,    0,    0],
         [   1, 4209, 5310,  ...,    0,    0,    0],
         [   1,    3,    0,  ...,    0,    0,    0],
         ...,
         [   1, 4209, 4209,  ...,    0,    0,    0],
         [   1,    3,    0,  ...,    0,    0,    0],
         [   1, 4165, 5628,  ...,    0,    0,    0]])}

In [16]:
next(d)["ids"]

tensor([[   1, 4165, 9834,  ...,    0,    0,    0],
        [   1, 4184, 1272,  ...,    0,    0,    0],
        [   1, 4184, 1272,  ...,    0,    0,    0],
        ...,
        [   1, 1272, 7938,  ...,    0,    0,    0],
        [   1,    3,    0,  ...,    0,    0,    0],
        [   1, 4209, 4209,  ...,    0,    0,    0]])